In [8]:
import numpy as np
import math
import networkx as nx
from pyvis.network import Network

In [11]:
class louvain_ascendency:
    
    
    def __init__(self, network):
        self.network = network
        x, y = self.network.shape
        self.community_ids = []
        self.partitioning = {}
        self.ascendencies = {}
        for node_id in range(x):
            self.community_ids.append(node_id)
            self.partitioning[node_id] = list()
            self.partitioning[node_id].append(node_id)
            self.ascendencies[node_id] = 0
        
    def optimise(self):
        x, y = self.network.shape
        move = True

        while(move == True):

            move = False
            for i in self.partitioning:
                neighbours = self.get_neighbour_ids(i)
                amis = []
                used_neighbours = []
                for j in neighbours:
                    if(i != self.community_ids[j]):
                        amis.append(self.test_union(i, self.community_ids[j]))
                        used_neighbours.append(j)
                if(len(amis) != 0):
                    max_ami = max(amis)
                else:
                    max_ami = -1
                    
                old_ami_id1 = self.test_union(i, i)
                old_ami_id2 = self.test_union(j, j)
               
                
                if((max_ami > old_ami_id1 and max_ami > old_ami_id2)):
                        
                    max_ami_id = amis.index(max_ami)
                    community_id_to_delete = self.community_ids[used_neighbours[max_ami_id]]
                    self.ascendencies[i] = max_ami
                    self.partitioning[i] = self.partitioning[i] + (self.partitioning[self.community_ids[used_neighbours[max_ami_id]]])
                    for node in self.partitioning[self.community_ids[used_neighbours[max_ami_id]]]:
                        self.community_ids[node] = i
                    self.partitioning[community_id_to_delete] = []
                    self.ascendencies[community_id_to_delete] = 0
                    move = True
        
        clean_partition = self.partitioning.copy()
        clean_ascendencies = self.ascendencies.copy()
        
        for community_id in self.partitioning:
            if(len(self.partitioning[community_id]) == 0):
                    del clean_partition[community_id]
                    del clean_ascendencies[community_id]
              
        
        return clean_partition, clean_ascendencies
        
    def test_union(self, id1, id2):
        if(id1 == id2):
            community_id = self.community_ids[id1]
            size = len(self.partitioning[community_id])
            new_network = np.zeros((size,size))
            all_ids = self.partitioning[community_id]
        else:
            community_id1 = self.community_ids[id1]
            community_id2 = self.community_ids[id2]
            size = len(self.partitioning[community_id1]) + len(self.partitioning[community_id2])
            new_network = np.zeros((size,size))
            all_ids = self.partitioning[community_id1] + self.partitioning[community_id2]
           
    
        for count1, i in enumerate(all_ids):
            for count2, j in enumerate(all_ids):
                new_network[count1][count2] = self.network[i][j]
                
        return measure_average_mutual_information(new_network)
        
        
        
    def get_neighbour_ids(self, community_id):
        x, y = self.network.shape
        ids = []
        for j in self.partitioning[community_id]:
            for i in range(x):
                if((self.network[j][i] != 0 or self.network[i][j] != 0) and i != j and (i not in ids)):
                    ids.append(i)
        return ids

def measure_average_mutual_information(network):
    
    N = network 
    Too = N.flatten().sum()
    AMI = 0
    column_sums = network.sum(axis=0)
    row_sums = network.sum(axis=1)
    x, y = network.shape
    for i in range(x):
        for j in range(y):
            a = network[i][j] * Too / (row_sums[i] * column_sums[j])
            if(a > 0):
                AMI += network[i][j] / Too * math.log10(a)
    return AMI

In [12]:

tree_matrix =  np.array([[0, 1, 1, 0, 0, 0, 0],
                         [1, 0, 0, 1, 1, 0, 0],
                         [1, 0, 0, 0, 0, 1, 1], 
                         [0, 1, 0, 0, 0, 0, 0], 
                         [0, 1, 0, 0, 0, 0, 0],
                         [0, 0, 1, 0, 0, 0, 0],
                         [0, 0, 1, 0, 0, 0, 0]])

network = tree_matrix

G = louvain_ascendency(network)
print(G.optimise())


({6: [6, 5, 4, 3, 2, 0, 1]}, {6: 0.5017166594399687})


C:\Users\Ivan\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:104: RuntimeWarning: invalid value encountered in double_scalars
